In [ ]:
import torch
from PIL import Image
import cv2

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Load an image
img = cv2.imread('/content/images/chemed/photo_2022-09-05_09-57-09 (1).jpg')

# Perform object detection
results = model(img)

# Display results
results.show()

In [ ]:
# Extract bounding box coordinates, confidence scores, and class labels
detections = results.xyxy[0]  # assuming single image
for *xyxy, conf, cls in detections:
    print(f"Coordinates: {xyxy}, Confidence: {conf}, Class: {cls}")

In [ ]:
from sqlalchemy import create_engine, Column, Integer, Float, String, MetaData, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

class Detection(Base):
    __tablename__ = 'detections'
    id = Column(Integer, primary_key=True)
    x1 = Column(Float)
    y1 = Column(Float)
    x2 = Column(Float)
    y2 = Column(Float)
    confidence = Column(Float)
    class_label = Column(String)

# Create an engine and session
engine = create_engine('sqlite:///detections.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

# Save detections to the database
for *xyxy, conf, cls in detections:
    detection = Detection(x1=xyxy[0], y1=xyxy[1], x2=xyxy[2], y2=xyxy[3], confidence=conf.item(), class_label=cls.item())
    session.add(detection)
session.commit()

In [ ]:
import logging

# Configure logging
logging.basicConfig(filename='detection.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Log the start of the detection process
logging.info('Started object detection process')

try:
    # Your object detection and data processing code here
    pass
    logging.info('Object detection process completed successfully')
except Exception as e:
    logging.error(f'Error during object detection process: {e}')